In [2]:
#Importing Required Libraries 
import os
import pandas as pd
import numpy as np

In [3]:
#Data Loaders 
DATA_FOLDER = "nse_prices"

FY_START = pd.to_datetime("2020-04-01")
FY_END   = pd.to_datetime("2025-03-31")

In [6]:
#Importing CSV with Data on Shares Outstanding as on 31-03-2025
shares_df = pd.read_csv("shares_outstanding.csv")

shares_outstanding = dict(zip(shares_df["Bank"], shares_df["Shares Outstanding"]))
shares_outstanding

{'SBIBANK': 8924620034,
 'BANKBARODA': 5171362179,
 'CANBK': 9076562500,
 'HDFCBANK': 5105325797,
 'ICICIBANK': 3564037855,
 'AXISBANK': 3098620347,
 'KOTAKBANK': 1988519342,
 'INDUSINDBK': 779445161,
 'BAJFINANCE': 6208203435,
 'PNB': 11521086957}

In [8]:
#Helper Function to get Last Traded Price of Stock in FY25
def get_fyend_price(df, fy_end=FY_END):
    df = df.sort_values("Date")
    df = df[df["Date"] <= fy_end]
    if df.empty:
        return None, None
    last_row = df.iloc[-1]
    return last_row["Adj Close"], last_row["Date"]

In [10]:
results = []
missing = []

for fname in os.listdir(DATA_FOLDER):
    if not fname.lower().endswith(".csv"):
        continue
    
    ticker = fname.replace(".csv", "")
    file_path = os.path.join(DATA_FOLDER, fname)

    df = pd.read_csv(file_path)

    # Convert and remove timezone
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df["Date"] = df["Date"].dt.tz_localize(None)

    # Extract FY-end adjusted close
    fy_close, price_date = get_fyend_price(df)
    
    if fy_close is None:
        print(f"⚠ No FY-end price found for {ticker}")
        continue
    
    if ticker not in shares_outstanding:
        print(f"⚠ Shares outstanding missing for {ticker} in shares_outstanding.csv")
        continue
    
    shares = shares_outstanding[ticker]
    market_cap = fy_close * shares
    
    results.append({
        "Ticker": ticker,
        "AdjClose_FYend": fy_close,
        "Price_Date": price_date.date(),
        "Shares_Outstanding": shares,
        "Market_Cap_Rupees": market_cap,
        "Market_Cap_Crore": market_cap / 1e7
    })

marketcap_df = pd.DataFrame(results).sort_values("Ticker").reset_index(drop=True)
marketcap_df

,Ticker,AdjClose_FYend,Price_Date,Shares_Outstanding,Market_Cap_Rupees,Market_Cap_Crore
0,AXISBANK,1101.058960,2025-03-28,3098620347,3.411764e+12,341176.369658
1,BAJFINANCE,889.073914,2025-03-28,6208203435,5.519552e+12,551955.172422
2,BANKBARODA,220.916595,2025-03-28,5171362179,1.142440e+12,114243.972647
3,CANBK,85.921478,2025-03-28,9076562500,7.798717e+11,77987.166762
4,HDFCBANK,901.908936,2025-03-28,5105325797,4.604539e+12,460453.895519
5,ICICIBANK,1338.025635,2025-03-28,3564037855,4.768774e+12,476877.401327
6,INDUSINDBK,649.849976,2025-03-28,779445161,5.065224e+11,50652.241885
7,KOTAKBANK,2168.699463,2025-03-28,1988519342,4.312501e+12,431250.082894
8,PNB,93.422897,2025-03-28,11521086957,1.076333e+12,107633.332402
9,SBIBANK,756.313538,2025-03-28,8924620034,6.749811e+12,674981.094963


In [11]:
marketcap_df.to_csv("marketcap_FY2025.csv", index=False)